In [21]:
from dilithium import Dilithium2
from dilithium import Dilithium3
from dilithium import Dilithium5
from datetime import datetime

In [34]:
# Example of signing
#pk, sk = Dilithium2.keygen()
#msg = b"Hello"
#sig = Dilithium2.sign(sk, msg)
#Dilithium2.verify(pk, msg, sig)

# Generación de claves para CA
def generate_ca_keypair():
    public_key, private_key = Dilithium2.keygen()
    return private_key, public_key

# Generación de claves para certificados
def generate_cert_keypair():
    public_key, private_key = Dilithium2.keygen()
    return private_key, public_key

# Funciones de firma

def generate_signature(private_key, data):
    datos= bytes(data, 'utf-8')
    sk=private_key
    firma = Dilithium2.sign(sk, datos)
    # Simula la generación de una firma utilizando la clave privada
    return firma

# Verificación de firma
def verify_signature(public_key, data, firma):
    pk=public_key
    datos= bytes(data, 'utf-8')
    return Dilithium2.verify(pk, datos, firma)


In [11]:
# Crear Autoridad Certificadora (CA)
def Crear_Autoridad():
    #nivel_S=512 
    ca_private_key, ca_public_key = generate_ca_keypair()
    print("Escriba nombre de la autoridad")
    nombre_Autoridad=input(str())
    print("Escriba el email")
    email           =input(str())
    print("Pais de origen")
    pais            =input(str())
    data_Autoridad={
        'Autoridad':nombre_Autoridad,
        'País':pais,
        'e-mail':email
    }
    #print(data_Autoridad,ca_private_key, ca_public_key)
    return data_Autoridad,ca_private_key, ca_public_key

In [14]:
# Solicitud de certificado
def Solicitud_Certificado():
    #nivel_S=512
    cert_private_key, cert_public_key = generate_cert_keypair()
    print("Nombre solicitante")
    nombre=input(str)
    
    cert_data = {
    'subject': nombre,
    'public_key': cert_public_key}
    #print(cert_data)
    return cert_data,cert_private_key

In [5]:
def Generacion_Certificado_Autoridad(solicitud_Certificado,Autoridad):
    datos=Autoridad[0]
    llave_privada=Autoridad[1]
    # Firma del certificado por la CA
    certificado_datos=solicitud_Certificado
    certificado_datos['Fecha']=datetime(2025, 1, 1)
    certificado_firmado= generate_signature(llave_privada, str(certificado_datos))
    #print(certificado_firmado,certificado_datos)
    return certificado_firmado,certificado_datos

In [31]:
def Frame(mensaje,certificado,llave_privada):
    sk=llave_privada
    mensaje_=bytes(mensaje, 'utf-8')
    
    #firma=sk.sign(mensaje_)
    firma=generate_signature(sk, mensaje)
    frame={'Certificado_Datos':certificado[1],
           'Firma_del_Certificado':certificado[0],
           'Mensaje':mensaje,
           'Mensaje_firmado':firma}
    #print(frame)
    return frame

In [7]:
def Verificacion_Certificado(ca_public_key,Frame):
    certificado_datos=Frame['Certificado_Datos']
    firma_certificado=Frame['Firma_del_Certificado']
    mensaje=Frame['Mensaje']
    mensaje_firmado=Frame['Mensaje_firmado']
    llave_publica_solicitante=certificado_datos['public_key'] #Llave publica del que manda el msj
    # Verificar la firma de la CA en el certificado
    ca_verification_result = verify_signature(ca_public_key, str(certificado_datos),firma_certificado)
    if ca_verification_result:
        print("Firma de la CA verificada correctamente en el certificado.")
        fecha_expiracion=certificado_datos['Fecha']
        hoy = datetime.now()
        if hoy<fecha_expiracion:
            
            if verify_signature(llave_publica_solicitante,str(mensaje),mensaje_firmado):
                print("Firma del certificado verificada")
                return True
            
            
        else:
            print("El certificado ha expirado")
            return False
    else:
        print("Firma de la CA no válida en el certificado.")
        return False

In [12]:
Autoridad=Crear_Autoridad()

Escriba nombre de la autoridad
CA
Escriba el email
ca.email@.com
Pais de origen
Mexico


In [19]:
sc,llave_privada=Solicitud_Certificado()

Nombre solicitante
<class 'str'>Eliu


In [22]:
GS=Generacion_Certificado_Autoridad(sc,Autoridad)

In [32]:
frame= Frame("mensaje",GS,llave_privada)

In [35]:
verificacion=Verificacion_Certificado(Autoridad[2],frame)

Firma de la CA verificada correctamente en el certificado.
Firma del certificado verificada


In [36]:
Autoridad

({'Autoridad': 'CA', 'País': 'Mexico', 'e-mail': 'ca.email@.com'},
 b'mo]\x89$0\x186S\x8af9_\xf4R9\xc8\xbd\x1a\xe7\x00\xea\x91\x86?\xde\xe4\xc9\xde\xdd\xc3G\xbc\xb6/\xfa\xe3\xc8\xed\xd7C5E\xa2Z\xa9\xc9}\xa4\xdeL\x8e%L\xe0\xf3\x1f{\x10?\xa2\x8e\xda\xce\x1c\xf4u\xab\x9d\x11-\x884\t$\xcf\x03\xfe\xe7\xc6\xcfZ\xe0\x8f.\x18cu\x107\'\xdd\x9c\x8fz\x93XF\x11L\x92\x08\x0cB\x81\xa2\xa0m\x04A\x85\\\x06M\x11)\x90A\xb4\r\x1c\'!\xc3\x82@\xc0"\x88$\xa2\x88\x90\x94\x05\x93\xa8\x88"\x990\xa1\x08n\x1b8B\x1b8F!3\n\x0b\x99`B\xc8 \xd1\xc8lJ4\x12\xc8\xa4Q\xdaB\x00\x01&$\x03\xa8m!\xb2\x89\x00\xb4\x89\xd3\xb2 \x00\xb2L\x01\x05R\xcc\x18$\x11$h\x109)\x91BfS\x06&\n8F\\\xa6$ \x81\x08\x1c\xb5\x81L\x16f\x12H\x85\x8a(\x89\x19\xc7\x00\xc80\x01R\x88\x89Y\xa2@\xc4\x00\x12\x8c\x14mP\x14e\xe3(`\x1b\x10-\xe1\xc6p\x92\xb6e\xa2\x06\x82"\xc7\x10\x1b\x80\x00\x02\xa0Pa\x84\tP\xc2M\x1b%"\x9b Q\x99\x92\x10\n\'"\xc2\x16m\xc1@ !\x90IY&%\x90DB\x0c\xb2q`D@ \xa0\r\x19\xb3\x04\x80\xa8\x11P&"\x1a\x99,\x10\x01\x01\xe3\x92-QF\x84\x80\x84)

In [37]:
sc,llave_privada

({'subject': 'Eliu',
  'public_key': b'|\xbdo\xc2\xa7\xbaLT^L\xc4Q\xe2@)\x1e(S\x18\x0e\xb2\x86i5\xcf\xec\xac\x07\xf3fx\xba\x9c\x80\xd4n\x8f+\x05\xd9/\x83\xb1\x14\xe6\x15\x16\xaa\xf2\xed7\xe3)\x04\xa2\xe91\x10m\xcb\x99~\x80\x08\xb5\xde\xa1\xe5eNf&\xf9\x14\rcGu\xbc\x9c\xc6\xc9\xb3\xe5\x12E\x06\xc3\x10\xee\xbe\xf1\xe2B\xee|%\x80\xce\xdb\x84\xeee\x93\rf\xd4\xfey\xa3V\xb4\x9d\xa2a\x8e\x14\xf4\x83\xd3o\x988C\xf6}\xa5\xa9n3\x9d\x17\x98\x8f\xf6\x0cu\x00\x1c+\x838\xac\x17)\xc9\x97\xf9\xcd\xf1~\xbc\xbb\xb3\xad\x86\x9b\xd8\xf6*e\xb9E\xbfBn\xd1\xed\x9b4M\xec\xe3t!\xe7\xd9\xed\x8e5G\x1c\xc7\x01\x9f|\xc7\x10\x96\xe8J`@\xae\xd9a+3?~\xealR;d\x80U\xba\xa0aMn\xc2\x9aA%Y\x16\xef\x00\x02\xb1h~\x9a\x9cM\x8b"1\xbcM\xba\xfb=M+3=\xd7\xaf\x88o\xb3\'\x04\x1ap\x94\xc3\x96\x06\xd5cc\x89\xb4\xbb!d,\xac\xa3S\xe6\xe6\xaa\xe6\xd1\xd0:\x96\x93\xd2\x04{;\xbd\x7f~\xa8\xe5\xb6p)\x80"\x92p%\x1c\xdb\x08\xaaA\xf3:z\xd5\x9cA\x1dF\x93tg\xca\x04[\x92c\xef\xa4\xab\xf4\xeb\xf0\x99\xd0\xf9\'\xe5\xaeq\xcaW@\xda\xd7\x10\'\xd6\xb8A7

In [38]:
GS

(b'\xfcK\x83\xfft\xb1s\xdd\xf5$\xf4\xc1\x95\xb9\xd5\xe56\xc1\xd4\x86r{\x80L}7\xe1#\xb4\x9b@&\xfb\xf5]\xc3:D\x92t|\x91\x11\xa8\x1e\xfe\xffYL\x02ey\xe6/i\x9e\xd1\xf9S\x0eG\xc9y\x1f\x18n\xd0t\xadG A$\xe4\xdd\xa6\xe2\xa0\x9f=<\xa5\x06 \r\xa1b?\xcb\xb94\xefB\xbb\xa9\x11EE4\x94f\xf0=<\r\x9aXS\xdf\x85\x817_8\xf7\x02&|],!\x80\xe5\xf3\xc1\xa2\xd9\x8a\x02\x84$\x14\xd7<\x02g+\x9d\x830\xa0\x1a\xa1\x05\x16\x18\x8d\xba"\xadx\xac\xa0o\xcd\xd5\xa3\x95Tz\x98\xa4\x13\x049\xbd\x90m\xe3\x01y\xe674\xaba\xe78\xef\xfa3$A\x9fM\xea\xe1\xbd\x90\xe5[\x99\x95\xbbVp.\xc7\x1d\rx\xed\xcd\xc3\x18\xd55\xf1\xc7\x81\xd1\xec\xcbM6F\x8d\x8c\x8d\xee \xcck\xf5\xd1\n5\x0f\x85\xc3+.n\x17\x99\xec\xa4\x10\x89\x1f\x9f\xf9\xaa\x07\xaf\x88\xcc^\\\xe9\x99-\xd3\xa6\x9b\x02\x01\x94K\xfc\xae5\xd5W\xb4\xe5\xc8\x1bj\x976\xe5>\xe0\x01L \x93\xad\xa8e\xdf\xebT\xec\xe7k\xec\xae,\xf812\xd4\xa3\xe9\xd2\xff\xb8\xbb\xbc"\xf6%\x18\xd55\xae\x0c\xe1G\xdb\xd44\x9a\x87\xf9\x99V>\xd6\x8e\x15\xb9\xa7&\xdb@\xa7\xb4Cq\xd4\xe9E47%\x05\xf9\x1e\xfc\xc1\x98

In [39]:
frame

{'Certificado_Datos': {'subject': 'Eliu',
  'public_key': b'|\xbdo\xc2\xa7\xbaLT^L\xc4Q\xe2@)\x1e(S\x18\x0e\xb2\x86i5\xcf\xec\xac\x07\xf3fx\xba\x9c\x80\xd4n\x8f+\x05\xd9/\x83\xb1\x14\xe6\x15\x16\xaa\xf2\xed7\xe3)\x04\xa2\xe91\x10m\xcb\x99~\x80\x08\xb5\xde\xa1\xe5eNf&\xf9\x14\rcGu\xbc\x9c\xc6\xc9\xb3\xe5\x12E\x06\xc3\x10\xee\xbe\xf1\xe2B\xee|%\x80\xce\xdb\x84\xeee\x93\rf\xd4\xfey\xa3V\xb4\x9d\xa2a\x8e\x14\xf4\x83\xd3o\x988C\xf6}\xa5\xa9n3\x9d\x17\x98\x8f\xf6\x0cu\x00\x1c+\x838\xac\x17)\xc9\x97\xf9\xcd\xf1~\xbc\xbb\xb3\xad\x86\x9b\xd8\xf6*e\xb9E\xbfBn\xd1\xed\x9b4M\xec\xe3t!\xe7\xd9\xed\x8e5G\x1c\xc7\x01\x9f|\xc7\x10\x96\xe8J`@\xae\xd9a+3?~\xealR;d\x80U\xba\xa0aMn\xc2\x9aA%Y\x16\xef\x00\x02\xb1h~\x9a\x9cM\x8b"1\xbcM\xba\xfb=M+3=\xd7\xaf\x88o\xb3\'\x04\x1ap\x94\xc3\x96\x06\xd5cc\x89\xb4\xbb!d,\xac\xa3S\xe6\xe6\xaa\xe6\xd1\xd0:\x96\x93\xd2\x04{;\xbd\x7f~\xa8\xe5\xb6p)\x80"\x92p%\x1c\xdb\x08\xaaA\xf3:z\xd5\x9cA\x1dF\x93tg\xca\x04[\x92c\xef\xa4\xab\xf4\xeb\xf0\x99\xd0\xf9\'\xe5\xaeq\xcaW@\xd